# Unet Model Prediction

 - Produce mask images on test dataset
 - Get foreground pixels for mask images and save as pickle file

## Load Images and Make Prediction

In [1]:
import tensorflow as tf
import numpy as np

# load saved model
model = tf.keras.models.load_model('unet_model.h5')

In [2]:
def create_mask(input_image):
  # get current image's size
  w, h, _ = input_image.shape

  # prepare image for model's prediction
  input_image = tf.image.resize(input_image, (128, 128))     # resize image
  input_image = tf.cast(input_image, tf.float32) / 255.0     # normalize image
  input_image = input_image[None,:,:]

  # get predicted mask
  pred_mask = tf.argmax(model.predict(input_image), axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]

  # resize image back to original size
  out_image = tf.image.resize(pred_mask[0], (w, h))

  return out_image

In [3]:
from zipfile import ZipFile
from skimage.io import imread

# read input images from a zip file and produce predicted mask
def loadData(filepath):
    predicted_masks = []   # store predicted mask images
    
    with ZipFile(filepath, 'r') as zipdata:
        namelist = zipdata.namelist()
        
        # loop through every directory/file
        for file_path in namelist:
            # read image files and save to list
            if '.jpg' in file_path or '.JPG' in file_path:
                # load image into list
                img_bytes = zipdata.open(file_path)
                image = np.array(imread(img_bytes))            # read image byte from zip file
                predicted_masks.append(create_mask(image))     # get predicted mask image
    
    # close zip file
    zipdata.close()
    
    # return a list of predicted mask images
    return predicted_masks

In [4]:
# load input images and get predicted masks
masks = loadData('../data/Oxford-IIIT Pet test.zip')

## Get Foreground Pixels

In [5]:
def get_foreground(segmented_image):
  np_image = segmented_image.numpy()
  foreground_pixels = []
  foreground_value = 0

  # iterate through height
  for y in range(segmented_image.shape[0]):
    # iterate through width
    for x in range(segmented_image.shape[1]):
      # if this is a foreground pixel
      if np_image[y, x] == foreground_value:
        foreground_pixels.append((x, y)) # add it to my list of (x, y) coordinate pairs
  
  return foreground_pixels

In [6]:
fg_pixels = []      # foreground pixels of predicted mask

for predicted_mask in masks:
  fg_pixels.append(get_foreground(predicted_mask))

In [7]:
import pickle

# save foreground pixels
with open('../data/model_fg_pixels.pkl', 'wb') as f:
  pickle.dump(fg_pixels, f)